In [ ]:
prefix = "lidar-data"
bucket = "sagemaker-us-east-1-224863548990"

In [ ]:
from sagemaker import get_execution_role
import boto3
try: 
    role = get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_role')['Role']['Arn']

In [ ]:
client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = "pytorch-extending-our-containers-cifar10-example"

ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

In [ ]:
data_location = "s3://{}}/{}".format(bucket,prefix) 

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {"epochs": 1}

instance_type = "ml.m4.xlarge"

estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_uri=ecr_image,
    hyperparameters=hyperparameters,
)

estimator.fit(data_location)

predictor = estimator.deploy(1, instance_type)

In [ ]:
import torchvision, torch
import numpy as np

from utils.utils_cifar import get_test_data_loader, imshow, classes
testloader = get_test_data_loader("/tmp/lidar")

from sagemaker.predictor import json_serializer, json_deserializer

In [ ]:
# get some test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print("GroundTruth: ", " ".join("%4s" % classes[labels[j]] for j in range(4)))

# predictor.accept = "application/json"
# predictor.content_type = "application/json"

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

outputs = predictor.predict(images.numpy())

_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print("Predicted: ", " ".join("%4s" % classes[predicted[j]] for j in range(4)))

In [ ]:
predictor.delete_endpoint()